In [ ]:
# Imports 

!pip install bayesian-optimization

import pandas as pd
from xgboost import XGBClassifier, cv
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from bayes_opt import BayesianOptimization
import numpy as np

In [ ]:
# Set SEED to define random state of any randomised functions
SEED=0
# Use seaborn theme 
sns.set_theme()

# Initial

## Loading the dataset

In [ ]:
def load_dataset_dragon():
    # load dragon descriptors
    dataset='Dragon'
    data_path = './data/alvaDescDescriptors.txt'
    na_values=['na']
    df = pd.read_csv(data_path, sep="\t",  dtype={'pctapi': np.float64}, na_values=na_values)
    smiles =  np.loadtxt("./data/smiles.txt", dtype='str')
    df = df.drop(['No.', 'NAME'], axis=1)
    df = df.set_index(smiles)
    
    # extract odor classes from the Modred dataset
    data_path = './data/master_4Mayhew.xlsx'
    df_mor, _, _ = load_dataset_modred()
    y=df_mor['label'] # labels, independent variables
    X=df # features, dependent variables
    X.columns = X.columns.str.translate("".maketrans({"[":"{", "]":"}","<":"^"}))

    return df, X, y

def load_dataset_modred():
    dataset='Modred'
    data_path = './data/master_4Mayhew.xlsx'
    df = pd.read_excel(data_path)
    df.set_index('SMILES', inplace=True)
    y=df['label'] # labels, independent variables
    X=df.drop(['label'], axis=1) # features, dependent variables

    return df, X, y

## Initial stats

In [ ]:
def initial_df_stats(df):
    print(f"Number of columns with all NaNs \n{(df.isna().mean(axis=0) == 1).value_counts()}")
    print(f"\n Number of columns with >0.9 NaNs \n{(df.isna().mean(axis=0) > 0.9).value_counts()}")
    print(f"\n Number of columns with a NaN \n{ len(df.columns[df.isna().any()].tolist())}")
    
    percs = df.isna().mean(axis=0)
    ax = percs[(percs > 0)].plot.hist(bins=15)
    plt.xlabel("Percentage NaNs in descriptors with missing values")
    plt.show()
    
    df.describe()
    
    corrs = df.corr()
    corrs.to_csv(f'results/correlations/descriptor-correlations-{dataset}.csv')  

In [ ]:
def corrs_important_features(model, X_test, model_name, dataset, nlargest=100, heatmap=14):
    feature_imp = pd.Series(model.feature_importances_,index=X_test.columns.values).sort_values(ascending=False)
    top_100 = feature_imp.nlargest(100)
    important_df = df.loc[:,top_100.index]
    corrs = important_df.corr()
    
    print(f"Extracted correlation scores for most important {nlargest} features for {model_name}")
    corrs.to_csv(f'results/correlations/descriptor-corrs-top{nlargest}-{model_name}-{dataset}.csv')  
    
    # Print heatmap for top n
    print(f"Printing heatmap for most important {heatmap} features")
    
    top = feature_imp.nlargest(heatmap)
    important_df = df.loc[:,top.index]
    corrs = important_df.corr()
    
    plt.figure(figsize = (16,16))
    heatmap = sns.heatmap(corrs_10, vmin=-1, vmax=1, annot=True)
    # Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
    heatmap.set_title(f'Correlation Heatmap {dataset} - top {heatmap} descriptors', fontdict={'fontsize':12}, pad=12);

## Splitting the dataset 

In [ ]:
def split_dataset(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0) # 70% training and 30% test
    percentage_odorless = y_train.value_counts()[False]/y_train.shape[0]
    percentage_odor = 1 - percentage_odorless

    print(f"Total number molecules in training set: {y_train.shape[0]}")
    print(f"Odor: {y_train.value_counts()[True]}")
    print(f"Odorless: {y_train.value_counts()[False]}")

    print(f"\nTotal number molecules in test set: {y_test.shape[0]}")
    print(f"Odor: {y_test.value_counts()[True]}")
    print(f"Odorless: {y_test.value_counts()[False]}")

    print(f"\nPercentage odorless {y_test.value_counts()[False]/y_test.shape[0]}")
    
    X_train_t, X_val, y_train_t, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=SEED)

    print("Number of training samples:", len(X_train_t))
    print("Number of validation samples:", len(X_val))
    
    return X_train, X_test, X_train_t, X_val, y_train, y_test, y_train_t, y_val

## Preprocessing

In [ ]:
from sklearn import preprocessing 

# Scale dataset 

def preprocessing_df(df): 
    # Correlation
    corrs = df.corr()
    # select upper traingle of correlation matrix
    upper = corrs.where(np.triu(np.ones(corrs.shape),k=1).astype(bool))
    # Find index of columns with correlation greater than 0.99
    to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]
    # drop the columns
    df.drop(to_drop, axis=1, inplace=True)
    # Remove descriptors with all NaNs 
    df.dropna(axis=1, how='all', inplace=True)

def scale_dataset(X_train, X_test, X_train_t, X_val):

    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_transformed = min_max_scaler.fit_transform(X_train)

    X_train =  pd.DataFrame(X_train_transformed, columns=X_train.columns[0:], index=X_train.index)
    X_test = pd.DataFrame(min_max_scaler.transform(X_test), columns=X_test.columns[0:], index=X_test.index)

    min_max_scaler2 = preprocessing.MinMaxScaler()
    X_train_t_transformed = min_max_scaler2.fit_transform(X_train_t)

    X_train_t = pd.DataFrame(X_train_t_transformed, columns=X_train_t.columns[0:], index=X_train_t.index)
    X_val = pd.DataFrame(min_max_scaler2.transform(X_val), columns=X_val.columns[0:], index=X_val.index)

    return X_train, X_test, X_train_t, X_val

# SKLEARN helper functions

In [ ]:
def test_model(model, y_test, X_test, verbose=True):
    y_pred=model.predict(X_test)
    y_pred_probs=model.predict_proba(X_test)[:,1]
    
    if verbose:
        print("ROC_AUC (TEST):",metrics.roc_auc_score(y_test, y_pred_probs))
        print("\n\nCLASSIFICATION REPORT:\n",metrics.classification_report(y_test, y_pred,  digits=4))
        
        if hasattr(model, 'feature_importances_') and hasattr(X_test, 'columns'):
            feature_imp = pd.Series(model.feature_importances_,index=X_test.columns.values).sort_values(ascending=False)
            top_20 = feature_imp.nlargest(20)
            sns.barplot(x=top_20, y=top_20.index)
            plt.xlabel('Descriptor Importance Score')
            plt.ylabel('Descriptor')
            plt.title("Top descriptors")
            plt.show()
        
        fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_probs)  
        
        #create ROC curve
        plt.plot(fpr,tpr)
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
        
    return metrics.roc_auc_score(y_test, y_pred_probs)

def test_model_cross_val(model, y_test, X_test, k=5, verbose=True):
    roc_auc_nans = cross_val_score(model, X_test, y_test, cv=k, scoring='roc_auc')
    print(f"AVERAGE CV={k} ROC_AUC: {np.mean(roc_auc_nans)}")
    print(f"AVERAGE CV={k} STD: {np.std(roc_auc_nans)}")
    return np.mean(roc_auc_nans), np.std(roc_auc_nans)

def handle_importance(model, X_train, X_test=pd.DataFrame(), threshold=-1, n=-1, verbose=None):
    feature_imp = pd.Series(model.feature_importances_,index=X_train.columns.values).sort_values(ascending=False)
    if threshold >= 0:
        feature_imp = feature_imp[feature_imp > threshold]
    
    if n >= 0: 
        feature_imp = feature_imp.nlargest(n)
        
    if verbose: 
        print(feature_imp)
        
    X_train = X_train.loc[:, feature_imp.axes[0].tolist()]
    
    if not X_test.empty:
        X_test = X_test.loc[:, feature_imp.axes[0].tolist()]

    return X_train, X_test, model 

def optimiseImportance(model, X_tr, y_tr, k=5, model_name="RF", verbose=True):
    
    imp_params = [
              {"threshold":-1, 'n':1},
              {"threshold":-1, 'n':2},
              {"threshold":-1, 'n':5}, 
              {"threshold":-1, 'n':10}, 
              {"threshold":-1, 'n':25}, 
              {"threshold":-1, 'n':50},
              {"threshold":-1, 'n':100}, 
              {"threshold":-1, 'n':200}, 
              {"threshold":-1, 'n':400},
              {"threshold":-1, 'n':X_tr.shape[1]}]
    
    sns.set(rc={'figure.figsize':(11.7,12.27)})
    feature_imp = pd.Series(model.feature_importances_,index=X_tr.columns.values).sort_values(ascending=False)
    top_50 = feature_imp.nlargest(50)
    sns.barplot(x=top_50, y=top_50.index)
    plt.xlabel('Descriptor Importance Score')
    plt.ylabel('Descriptor')
    plt.title("Top 50 descriptors")
    plt.show()

    print(f"Cross validated importance experiment with k={k}")
    
    imp_df = pd.DataFrame(columns=['Most important N descriptors', 'Average AUROC', 'Standard Deviation'])

    for imp_param in imp_params: 
        name = (f"Threshold:{imp_param['threshold']}N:{imp_param['n']}".replace("N:-1", '')).replace("Threshold:-1", '')
        
        X_tr_temp , _, _ = handle_importance(model, 
                                            X_tr.copy(), 
                                            threshold=imp_param['threshold'], 
                                            n=imp_param['n'])
                
        roc_auc_nans = cross_val_score(model, X_tr_temp, y_tr, cv=k, scoring='roc_auc')
        
        imp_df = imp_df.append({'Most important N descriptors': imp_param['n'], 
                               'Average AUROC':np.mean(roc_auc_nans), 
                               'Standard Deviation':np.std(roc_auc_nans)}, 
                                ignore_index = True)

    if verbose:
        fig = imp_df.plot(kind="bar", x="Most important N descriptors", y="Average AUROC", 
                          yerr="Standard Deviation", 
                          figsize=(8,5),
                          title=f"AUROC after filtering descriptors by importance CV={k} - {model_name} - {dataset}", legend=False)
        plt.ylim(0.80,1)
    return imp_df

def testWithoutImportantDescs(model, X_tr, y_tr, k=5, model_name="RF", verbose=True):
    imp_params = [
                  {"threshold":-1, 'n':5}, 
                  {"threshold":-1, 'n':10}, 
                  {"threshold":-1, 'n':25}, 
                  {"threshold":-1, 'n':50},
                  {"threshold":-1, 'n':100}, 
                  {"threshold":-1, 'n':200}, 
                  {"threshold":-1, 'n':400},
                  {"threshold":-1, 'n':600}]
    
    imp_df = pd.DataFrame(columns=['Most important N descriptors', 'Average AUROC', 'Standard Deviation'])

    roc_auc, std = test_model_cross_val(model, y_tr, X_tr)
    imp_df = imp_df.append({'Most important N descriptors': 0, 
                       'Average AUROC':roc_auc, 
                       'Standard Deviation':std}, 
                        ignore_index = True)

    
    for imp_param in imp_params: 
        name = (f"Threshold:{imp_param['threshold']}")

        feature_imp = pd.Series(model.feature_importances_,index=X_tr.columns.values).sort_values(ascending=False)
        feature_imp = feature_imp.nlargest(imp_param['n'])
        X_tr_temp = X_tr.drop(feature_imp.axes[0].tolist(), axis=1) 
        roc_auc_nans = cross_val_score(model, X_tr_temp, y_tr, cv=k, scoring='roc_auc')
        
        imp_df = imp_df.append({'Most important N descriptors': imp_param['n'], 
                               'Average AUROC':np.mean(roc_auc_nans), 
                               'Standard Deviation':np.std(roc_auc_nans)}, 
                                ignore_index = True)

    if verbose:
        fig = imp_df.plot(kind="bar",figsize=(8,5), x="Most important N descriptors", y="Average AUROC",  ylabel="Average AUROC",  yerr="Standard Deviation", title=f"AUROC after removing descriptors by importance CV={k}  - {model_name} - {dataset}", legend=False)
        plt.ylim(0.80,1)
        
    return imp_df


from sklearn.impute import KNNImputer

def handleNaNs(X_tr, X_te, option=1, thresh=0.6): 
    if option == 1:
        nans =  X.columns[X.isna().any()].tolist()
        X_tr.drop(nans, inplace = True, axis=1)
        X_te.drop(nans, inplace = True, axis=1)
    else: 
        nans = X.isna().mean(axis=0)
        # set threshold for percentage nans before we drop 
        X_thresh = nans[nans >= thresh]
        X_tr.drop(X_thresh.index, 
          axis=1, 
          inplace=True)
        X_te.drop(X_thresh.index, 
          axis=1, 
          inplace=True)
        
        if option == 3: 
            imputer = KNNImputer(n_neighbors=2)
            X_tr_temp = imputer.fit_transform(X_tr)
            X_te_temp = imputer.transform(X_te)
            
            X_tr =  pd.DataFrame(X_tr_temp, columns=X_tr.columns[0:], index=X_tr.index)
            X_te = pd.DataFrame(X_te_temp, columns=X_te.columns[0:], index=X_te.index)
    
    return X_tr, X_te


def optimiseNaNs(model, X_train, X_test, y_train, y_test, model_name="RF", option=2, k=5, verbose=True): 
    print(f"Cross validated missing values experiment with k={k}")
    
    percentages = [x/100 for x in range(0, 100, 10)] 
    nans_df = pd.DataFrame(columns=['Percentage', 'Average AUROC', 'Standard Deviation'])

    for perc in percentages: 
        name = str(perc)
        if perc == 0:
            X_train_cv, _  = handleNaNs(X_train.copy(), X_test.copy(), option=1)
        else:
            X_train_cv, _  = handleNaNs(X_train.copy(), X_test.copy(), option=option, thresh=perc)

        roc_auc_nans = cross_val_score(model, X_train_cv, y_train, cv=k, scoring='roc_auc')
        
        nans_df = nans_df.append({'Percentage': name, 
                                   'Average AUROC':np.mean(roc_auc_nans), 
                                   'Standard Deviation':np.std(roc_auc_nans)}, 
                                    ignore_index = True)

    if verbose:
        fig = nans_df.plot(kind="bar", x="Percentage", y="Average AUROC", ylabel="Average AUROC", 
                           yerr="Standard Deviation", title=f"AUROC after thresholding descriptors by % NaNs CV={k} - {model_name} - {dataset}", 
                           legend=False)
        plt.ylim(0.9,1)
        
    return nans_df


# Using PCA to reduce dimensionality 

from sklearn import decomposition 

def optimiseDimensions(clf, X_train, y_train): 
    pca_df = pd.DataFrame(columns=['Dimensions', 'Average AUROC', 'Standard Deviation'])

    roc_aucs = []
    stds = []
    pcas = [1, 2, 5, 10, 20, 50, 100]

    for n in pcas:
        pca = decomposition.PCA(n_components=n)
        pca_result = pca.fit_transform(X_train)
        clf.fit(pca_result, y_train)

        print("Validation performance of Random Forest after reducing dimensionality with PCA")
        roc_auc, std = test_model_cross_val(clf, y_train, pca_result, k=5, verbose=True)

        pca_df = pca_df.append({'Dimensions': n, 
                               'Average AUROC':roc_auc, 
                               'Standard Deviation':std}, 
                                ignore_index = True)

    clf.fit(X_train_rf, y_train_rf)
    print("Validation performance of Random Forest with all dimensions")
    roc_auc, std = test_model_cross_val(clf, y_train, X_train, k=5, verbose=True)
    roc_aucs.append(roc_auc)
    pca_df = pca_df.append({'Dimensions': X_train.shape[-1], 
                           'Average AUROC':roc_auc, 
                           'Standard Deviation':std}, 
                            ignore_index = True)

    fig = pca_df.plot(kind="bar", x="Dimensions", y="Average AUROC", ylabel="Average AUROC", 
                       yerr="Standard Deviation", title=f"AUROC after reducing dimensions using PCA CV=5 - RF - {dataset}", 
                       legend=False, figsize=(10, 10))

    plt.ylim(0,1)

# Quick tuned model 

In [ ]:
def randomForestTuned(X_train, X_test, y_train, y_test): 
    # Run me if you want the tuned model without running training
    clf=RandomForestClassifier(random_state=SEED) 

    # For readability
    y_train_rf = y_train.copy()
    y_test_rf = y_test.copy()
    X_train_rf = X_train.copy()
    X_test_rf = X_test.copy()

    # Handle NaNs
    X_train_rf, X_test_rf = handleNaNs(X_train.copy(), X_test.copy(), option=1)
    X_train_t_rf, X_val_rf = handleNaNs(X_train_t.copy(), X_val.copy(), option=1)

    # Fit model 
    clf.fit(X_train_rf, y_train_rf)

    # Handle features 
    X_train_rf, X_test_rf, clf = handle_importance(clf, 
                                                   X_train_rf, 
                                                   X_test_rf, 
                                                   threshold=-1, 
                                                   n=100)

    # Create tuned model
    params_tuned_rf_saved = {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 1, 'n_estimators': 199}
    tuned_rf = RandomForestClassifier(**params_tuned_rf_saved, random_state=SEED)
    tuned_rf.fit(X_train_rf, y_train_rf)
    
    return tuned_rf, X_train, X_test, y_train, y_test


def xgboostTuned(X_train, X_test, y_train, y_test): 
    # Run me if you want the tuned model without running training

    # for readibility 
    y_train_xgb = y_train.copy()
    y_test_xgb = y_test.copy()
    X_train_xgb = X_train.copy()
    X_test_xgb = X_test.copy()

    xgb = XGBClassifier(random_state=SEED)
    xgb.fit(X_train_xgb, y_train_xgb)

    # Handle NaNs
    X_train_xgb, X_test_xgb = handleNaNs(X_train.copy(), X_test.copy(), option=2, thresh=0.1)
    X_train_t_xgb, X_val_xgb = handleNaNs(X_train_t.copy(), X_val.copy(), option=2, thresh=0.1)

    # Fit model 
    xgb.fit(X_train_xgb, y_train_xgb)

    # Handle features 
    X_train_xgb, X_test_xgb, xgb = handle_importance(xgb, 
                                                    X_train_xgb, 
                                                    X_test=X_test_xgb, 
                                                    threshold=-1, 
                                                    n=400)

    X_train_t_xgb, X_val_xgb, xgb = handle_importance(xgb, 
                                                    X_train_t_xgb, 
                                                    X_test=X_val_xgb, 
                                                    threshold=-1, 
                                                    n=400)
    xgb.fit(X_train_xgb, y_train_xgb)

    # Create tuned model
    params_tuned_xgb_saved = {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 564, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'scale_pos_weight': 2.5}
    tuned_xgb.fit(X_train_xgb, y_train_xgb)
    
    return X_train, X_test, y_train, y_test


## Neural Network Helper Functions 

In [ ]:
def preprocessing_nn(X_train_nn, X_test_nn, X_train_t_nn, X_val_nn, y_train_nn, y_test_nn, y_train_t_nn, y_val_nn, ): 
    # - You can't have missing values in a Neural Network, so we choose to use the optimal value for XGB
    X_train_nn, X_test_nn = handleNaNs(X_train_nn.copy(), X_test_nn.copy(), option=3, thresh=0.1)
    X_train_t_nn, X_val_nn = handleNaNs(X_train_t_nn.copy(), X_val_nn.copy(), option=3, thresh=0.1)
    
    encoder = LabelEncoder()
    encoder.fit(y_train_t_nn)
    
    y_train_nn = encoder.transform(y_train_nn)
    y_train_t_nn = encoder.transform(y_train_t_nn)
    y_test_nn = encoder.transform(y_test_nn)
    y_val_nn = encoder.transform(y_val_nn)

def test_model_keras_train(model, history, X_val_nn, y_val_nn, X_train_nn, y_train_nn): 
    # Plot training and validation auc 

    plt.plot(history.history['auc'])
    plt.plot(history.history['val_auc'])
    plt.title('model roc_auc')
    plt.ylabel('ROC_AUC')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='upper left')
    plt.show()

    # Plot training and validation ROC curve
    y_pred_keras = model.predict(X_val_nn).ravel()
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val_nn, y_pred_keras)
    auc_v_keras = metrics.roc_auc_score(y_val_nn, y_pred_keras)

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras)

    # Use the Keras model to make predictions on a test dataset
    y_pred_t_keras = model.predict(X_train_nn)
    
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_train_nn, y_pred_t_keras)
    auc_t_keras = metrics.roc_auc_score(y_train_nn, y_pred_t_keras)
    
    plt.plot(fpr_keras, tpr_keras)

    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(['', 'Validation (area = {:.4f})'.format(auc_v_keras), 'Training (area = {:.3f})'.format(auc_t_keras)], loc='best')
    plt.show()
    plt.show()
    
    # Print classification report 
    y_pred = model.predict(X_val_nn).round()
    print(metrics.classification_report(y_val_nn, y_pred, digits=4))
    
def test_model_keras(model, X_test_nn, y_test_nn, X_train_nn, y_train_nn):
    y_pred_t_keras = model.predict(X_test_nn).ravel()
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test_nn, y_pred_t_keras)
    auc_t_keras = metrics.roc_auc_score(y_test_nn, y_pred_t_keras)
    auc_t_keras

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(['', 'Test (area = {:.4f})'.format(auc_t_keras)], loc='best')
    plt.show()
    plt.show()
    
    # Print classification report 
    y_pred = model.predict(X_test_nn).round()
    print(metrics.classification_report(y_test_nn, y_pred, digits=4))
    

def model_builder_2(hp):

    model = keras.Sequential()
    model.add(Dense((X_train_t_nn_temp.shape[-1]), input_shape=((X_train_t_nn_temp.shape[-1]),), activation='relu'))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 2-length of layers
    hp_units = hp.Int('unit_1', min_value=2, max_value=2000, step=10)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    # dropout layer reduces overfitting
    hp_units = hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)
    model.add(keras.layers.Dropout(hp_units))
    
    # Tune the number of units in the second Dense layer
    # Choose an optimal value between 2-length of layers
    hp_units = hp.Int('unit_2', min_value=2, max_value=2000, step=10)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=METRICS)

    return model
    

import sklearn.feature_selection as fs
from sklearn.svm import LinearSVC
import random

def feature_ranking_experiment(X_train_t_nn,X_val_nn, y_train_t_nn, y_val_nn ): 
    ranking = []
    algos = ['random', 'rf', 'f_classif', 'mutual_info_classif']
    ks = [5, 50, 100, 400]

    accuracy_2 = pd.DataFrame(index = ks, columns = algos)

    for algo in algos:
          for k in ks:
                X_train_t_nn_temp = X_train_t_nn.copy()
                X_val_nn_temp = X_val_nn.copy()

                if algo == 'random':
                    indexes = list(range(X_train_t_nn.shape[-1]))
                    random.shuffle(indexes)
                    top_n = indexes[0:k]
                    X_train_t_nn_temp = X_train_t_nn.iloc[:,top_n]
                    X_val_nn_temp = X_val_nn.iloc[:,top_n]
                else:
                    if algo == 'f_classif': 
                        bk = fs.SelectKBest(fs.f_classif, k=k)
                        bk.fit(X_train_t_nn, y_train_t_nn)
                    elif algo == "mutual_info_classif":
                        bk = fs.SelectKBest(fs.mutual_info_classif, k=k)
                        bk.fit(X_train_t_nn, y_train_t_nn)
                    elif algo == 'rf':
                        tuned_rf.fit(X_train_t_nn_temp, y_train_t_nn)
                        bk = fs.SelectFromModel(tuned_rf, prefit=True, max_features=k)

                    X_train_t_nn_temp = bk.transform(X_train_t_nn)
                    X_val_nn_temp = bk.transform(X_val_nn_temp)

                tuner = kt.Hyperband(model_builder_2,
                                     objective=kt.Objective("val_auc", direction="max"),
                                     max_epochs=10,
                                     factor=3,
                                     seed=SEED, 
                                     directory=None,
                                     project_name="odor",
                                     overwrite=True,)

                tuner.search(X_train_t_nn_temp, 
                             y_train_t_nn, 
                             epochs=30,
                             validation_data=(X_val_nn_temp, y_val_nn),
                             callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss")])

                # Get the optimal hyperparameters
                best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

                model_temp = tuner.hypermodel.build(best_hps)
                model_temp.summary()

                history = model_temp.fit(X_train_t_nn_temp, 
                                     y_train_t_nn, 
                                     epochs=30, 
                                     validation_data=(X_val_nn_temp, y_val_nn), 
                                     verbose=0)

                y_pred_keras = model_temp.predict(X_val_nn_temp).ravel()
                auc_keras = metrics.roc_auc_score(y_val_nn, y_pred_keras)
                accuracy_2.loc[k, algo] = auc_keras